In [ ]:
import json
import pandas as pd
import requests
from lattice import Connection


mode = 'prod'
connection = Connection(mode)
serv = connection.server

In [ ]:
obj_dict = {
    'human-postnatal-donors': 'HumanPostnatalDonor',
    'human-prenatal-donors': 'HumanPrenatalDonor',
    'tissues': 'Tissue',
    'tissue-sections': 'TissueSection',
    'organoids': 'Organoid',
    'cell-cultures': 'CellCulture',
    'suspensions': 'Suspension'
}

In [ ]:
def create_queries(ids):
    objects = {}
    blank = []
    for i in ids:
        objtype = i.split('/')[1]
        acc = i.split('/')[2]
        if objtype in objects:
            objects[objtype].append(acc)
        else:
            objects[objtype] = [acc]
    for k,v in objects.items():
        if v:
            url = serv + 'report/?type=' + obj_dict[k] + '&accession=' + '&accession='.join(v)
            if len(url) > 8000: #max url length is 8205
                mid = len(v)//2
                url1 = serv + 'report/?type=' + obj_dict[k] + '&accession=' + '&accession='.join(v[:mid])
                blank.append((str(len(v[:mid])) + ' ' + k, url1))
                url2 = serv + 'report/?type=' + obj_dict[k] + '&accession=' + '&accession='.join(v[mid:])
                blank.append((str(len(v[mid:])) + ' ' + k, url2))
            else:
                blank.append((str(len(v)) + ' ' + k, url))
    if blank:
        count = 0
        print(str(len(blank)) + ' URL(s)')
        for i in blank:
            count += 1
            print(f'{str(count)}: {i[0]}')
            print(i[1])

In [ ]:
def query(group, objects):
    if group.startswith('LATDS'):
        query = f'&dataset=/datasets/{group}/'
    else:
        query = f'&lab.name={group}'
    
    if objects == 'donors':
        query += '&field=donors.@id'
    elif objects == 'samples':
        query += '&field=derived_from.derived_from'
    elif objects == 'suspensions':
        query += '&field=derived_from.@id'
    
    url = serv + f'report/?type=Library{query}&limit=all'
    r = requests.get(url, auth=connection.auth).json()
    donors = []
    for l in r['@graph']:
        donors.extend(d['@id'] for d in l['donors']) #DIFFERENT
    create_queries(set(donors))

In [ ]:
def query_donors(group):
    if group.startswith('LATDS'):
        query = '&dataset=/datasets/' + group + '/'
    else:
        query = '&lab.name=' + group

    url = serv + f'report/?type=Library{query}&field=donors.@id&limit=all'
    r = requests.get(url, auth=connection.auth).json()
    donors = []
    for l in r['@graph']:
        donors.extend(d['@id'] for d in l['donors'])
    create_queries(set(donors))

In [ ]:
#currently, only retrieves the Suspensions that a Library derives from
#not a Suspension that another Suspension derives from
def query_suspensions_tissuesections(group):
    if group.startswith('LATDS'):
        query = '&dataset=/datasets/' + group + '/'
    else:
        query = '&lab.name=' + group

    url = serv + f'report/?type=Library{query}&field=derived_from.@id&limit=all'
    r = requests.get(url, auth=connection.auth).json()
    susps = []
    for l in r['@graph']:
        susps.extend(d['@id'] for d in l['derived_from'])
    create_queries(set(susps))

In [ ]:
def query_samples(group):
    if group.startswith('LATDS'):
        query = '&dataset=/datasets/' + group + '/'
    else:
        query = '&lab.name=' + group

    url = serv + f'report/?type=Library{query}&field=derived_from.derived_from&limit=all'
    r = requests.get(url, auth=connection.auth).json()
    samples = []
    for l in r['@graph']:
        for d in l['derived_from']:
            for dd in d['derived_from']:
                if dd.startswith('/suspensions/'):
                    second_susp = requests.get(serv + dd, auth=connection.auth).json()
                    samples.extend(second_susp['derived_from'])
                else:
                    samples.append(dd)
    create_queries(set(samples))

In [ ]:
#query Datasets by Accession
query_donors('LATDS210SIR')

In [ ]:
#query Datasets by Accession
query_samples('LATDS210SIR')

In [ ]:
#query Datasets by Accession
query_suspensions_tissuesections('LATDS210SIR')

In [ ]:
#query Labs by Name
query_donors('jason-spence')

In [ ]:
#query Labs by Name
query_samples('jason-spence')

In [ ]:
#query Labs by Name
query_suspensions_tissuesections('jason-spence')